In [7]:
import numpy as np
import pandas as pd
import tensorflow as tf
import sklearn

In [2]:
class FM(tf.keras.Model):
    def __init__(self, p, k=5):
        #p: dimension of each input
        #k: dimension of latent vector(feature)
        super(FM, self).__init__()

        self.w0 = tf.Variable([0.0])
        self.w = tf.Variable(tf.zeros([p]))
        self.v = tf.Variable(tf.random.normal(shape=(p, k)))

    def call(self, inputs):
        input_term = tf.reduce_sum(tf.math.multiply(self.w, inputs), axis=1)

        feature_interaction = 0.5 * tf.reduce_sum(
            tf.math.pow(tf.matmul(inputs, self.v), 2)
            - tf.matmul(tf.math.pow(inputs, 2), tf.math.pow(self.v, 2)),
            1,
            keepdims=False
        )

        y_pred = tf.math.sigmoid(self.w0 + input_term + feature_interaction)

        return y_pred

In [13]:
def train_on_batch(model, optimizer, accuracy, inputs, targets):
    with tf.GradientTape() as tape:
        y_pred = model(inputs)
        loss = tf.keras.losses.binary_crossentropy(from_logits=False, y_true=targets, y_pred=y_pred)
    
    gradient = tape.gradient(loss, model.trainable_variables)
    optimizer.apply_gradients(zip(gradient, model.trainable_variables))

    accuracy.update_state(targets, y_pred)

    return loss


# 반복 학습 함수
def train(epochs, X, Y, verbose = 0):
    X_train, X_test, Y_train, Y_test = train_test_split(X, Y, test_size=0.2, stratify=Y)
    
    model = FM(p=X.shape[1], k=10)

    train_ds = tf.data.Dataset.from_tensor_slices((tf.cast(X_train, tf.float32), tf.cast(Y_train, tf.float32))).shuffle(500).batch(8)

    test_ds = tf.data.Dataset.from_tensor_slices((tf.cast(X_test, tf.float32), tf.cast(Y_test, tf.float32))).shuffle(200).batch(8)

    optimizer = tf.keras.optimizers.SGD(learning_rate=0.01)
    accuracy = BinaryAccuracy(threshold=0.5)
    loss_history = []

    for i in range(epochs):
        for x, y in train_ds:
            loss = train_on_batch(model, optimizer, accuracy, x, y)
            loss_history.append(loss)

        if verbose == 1:
            print("STEP {:03d} mean loss: {:.4f}".format(i, np.mean(loss_history)))
            print("STEP {:03d} mean accuracy: {:.4f}".format(i, accuracy.result().numpy()))


    test_accuracy = BinaryAccuracy(threshold=0.5)
    for x, y in test_ds:
        y_pred = model(x)
        test_accuracy.update_state(y, y_pred)

    print("TEST accuracy: {:.4f}".format(test_accuracy.result().numpy()))


In [14]:
from sklearn.preprocessing import MinMaxScaler
from sklearn.datasets import load_breast_cancer
from sklearn.model_selection import train_test_split
from tensorflow.keras.metrics import BinaryAccuracy

scaler = MinMaxScaler()
file = load_breast_cancer()
X, Y = file['data'], file['target']
X = scaler.fit_transform(X)

n = X.shape[0]
p = X.shape[1]
k = 10
batch_size = 8
epochs = 10

train(50, X, Y, verbose = 0)

STEP 000 mean loss: 0.7140
STEP 000 mean accuracy: 0.6419
STEP 001 mean loss: 0.5945
STEP 001 mean accuracy: 0.7243
STEP 002 mean loss: 0.5465
STEP 002 mean accuracy: 0.7545
STEP 003 mean loss: 0.5161
STEP 003 mean accuracy: 0.7785
STEP 004 mean loss: 0.4940
STEP 004 mean accuracy: 0.7934
STEP 005 mean loss: 0.4765
STEP 005 mean accuracy: 0.8033
STEP 006 mean loss: 0.4618
STEP 006 mean accuracy: 0.8120
STEP 007 mean loss: 0.4492
STEP 007 mean accuracy: 0.8201
STEP 008 mean loss: 0.4380
STEP 008 mean accuracy: 0.8270
STEP 009 mean loss: 0.4280
STEP 009 mean accuracy: 0.8324
STEP 010 mean loss: 0.4188
STEP 010 mean accuracy: 0.8371
STEP 011 mean loss: 0.4104
STEP 011 mean accuracy: 0.8416
STEP 012 mean loss: 0.4026
STEP 012 mean accuracy: 0.8452
STEP 013 mean loss: 0.3954
STEP 013 mean accuracy: 0.8485
STEP 014 mean loss: 0.3886
STEP 014 mean accuracy: 0.8516
STEP 015 mean loss: 0.3823
STEP 015 mean accuracy: 0.8546
STEP 016 mean loss: 0.3763
STEP 016 mean accuracy: 0.8571
STEP 017 mean 